In [100]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e3/sample_submission.csv
/kaggle/input/playground-series-s4e3/train.csv
/kaggle/input/playground-series-s4e3/test.csv


# **Input data**

In [101]:
train_df = pd.read_csv('/kaggle/input/playground-series-s4e3/train.csv')

train_df.head()


,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,0,584,590,909972,909977,16,8,5,2274,113,140,1358,0,1,50,0.7393,0.4000,0.5000,0.0059,1.0000,1.0000,0.0,1.2041,0.9031,0.6990,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,1,808,816,728350,728372,433,20,54,44478,70,111,1687,1,0,80,0.7772,0.2878,0.2581,0.0044,0.2500,1.0000,1.0,2.6365,0.7782,1.7324,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,2,39,192,2212076,2212144,11388,705,420,1311391,29,141,1400,0,1,40,0.0557,0.5282,0.9895,0.1077,0.2363,0.3857,0.0,4.0564,2.1790,2.2095,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,3,781,789,3353146,3353173,210,16,29,3202,114,134,1387,0,1,40,0.7202,0.3333,0.3333,0.0044,0.3750,0.9310,1.0,2.3222,0.7782,1.4314,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,4,1540,1560,618457,618502,521,72,67,48231,82,111,1692,0,1,300,0.1211,0.5347,0.0842,0.0192,0.2105,0.9861,1.0,2.7694,1.4150,1.8808,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1


In [102]:
X_feat = train_df.iloc[:,1:-7].abs()
y_train = train_df.iloc[:,-7:]

#X_feat.head()
y_train.head()

,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,1
2,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1


# **Feature engineering & scoring top 15 features**

* Y_Average: Average y coordinate
* X_Average: Average x coordinate
* Volume: Volume of steel plate
* Shape_Irregularity: Rough estimate of the fault shape in comparison to a square

In [103]:
#Feature engineering
def feature_engineer(train_data):
    
    train_data['Y_Average'] = (train_data['Y_Maximum'] + train_data['Y_Minimum']) / 2
    train_data['X_Average'] = (train_data['X_Maximum'] + train_data['X_Minimum']) / 2
    train_data['Volume'] = train_data['Pixels_Areas'] * train_data['Steel_Plate_Thickness']
    train_data['Shape_Irregularity'] = (train_data['X_Perimeter'] * train_data['Y_Perimeter']) / train_data['Pixels_Areas']

feature_engineer(X_feat)
X_feat.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,Length_of_Conveyer,TypeOfSteel_A300,TypeOfSteel_A400,Steel_Plate_Thickness,Edges_Index,Empty_Index,Square_Index,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Y_Average,X_Average,Volume,Shape_Irregularity
0,584,590,909972,909977,16,8,5,2274,113,140,1358,0,1,50,0.7393,0.4000,0.5000,0.0059,1.0000,1.0000,0.0,1.2041,0.9031,0.6990,0.5000,0.0104,0.1417,909974.5,587.0,800,2.500000
1,808,816,728350,728372,433,20,54,44478,70,111,1687,1,0,80,0.7772,0.2878,0.2581,0.0044,0.2500,1.0000,1.0,2.6365,0.7782,1.7324,0.7419,0.2997,0.9491,728361.0,812.0,34640,2.494226
2,39,192,2212076,2212144,11388,705,420,1311391,29,141,1400,0,1,40,0.0557,0.5282,0.9895,0.1077,0.2363,0.3857,0.0,4.0564,2.1790,2.2095,0.0105,0.0944,1.0000,2212110.0,115.5,455520,26.001054
3,781,789,3353146,3353173,210,16,29,3202,114,134,1387,0,1,40,0.7202,0.3333,0.3333,0.0044,0.3750,0.9310,1.0,2.3222,0.7782,1.4314,0.6667,0.0402,0.4025,3353159.5,785.0,8400,2.209524
4,1540,1560,618457,618502,521,72,67,48231,82,111,1692,0,1,300,0.1211,0.5347,0.0842,0.0192,0.2105,0.9861,1.0,2.7694,1.4150,1.8808,0.9158,0.2455,0.9998,618479.5,1550.0,156300,9.259117


# **Feature scoring and picking top 15 features**

In [104]:
BestFeatures = SelectKBest(score_func=chi2, k=10)

fit = BestFeatures.fit(X_feat,y_train)

df_scores = pd.DataFrame(fit.scores_)
df_columns = pd.DataFrame(X_feat.columns)

f_Scores = pd.concat([df_columns,df_scores],axis=1)
f_Scores.columns = ['Specs','Score']  
f_Scores.sort_values(by=['Score'])
#f_Scores
features = f_Scores.nlargest(20,'Score')
print(features)
features = features['Specs'].tolist()


                    Specs         Score
7       Sum_of_Luminosity  8.038570e+09
29                 Volume  2.354268e+09
3               Y_Maximum  1.803073e+08
27              Y_Average  1.768450e+08
2               Y_Minimum  1.735340e+08
4            Pixels_Areas  7.798256e+07
5             X_Perimeter  2.904111e+06
0               X_Minimum  2.239500e+06
28              X_Average  1.824961e+06
1               X_Maximum  1.467741e+06
6             Y_Perimeter  1.232963e+06
13  Steel_Plate_Thickness  8.432787e+04
8   Minimum_of_Luminosity  6.799199e+04
10     Length_of_Conveyer  4.429580e+04
30     Shape_Irregularity  3.116606e+04
21             LogOfAreas  2.700060e+03
11       TypeOfSteel_A300  2.508646e+03
22            Log_X_Index  2.023562e+03
12       TypeOfSteel_A400  1.673341e+03
20   Outside_Global_Index  1.317566e+03


# **Scaling of features**

In [105]:
ss = StandardScaler()
X_scaled = ss.fit_transform(X_feat[features])
#X_train.head()

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_train, test_size=0.2, random_state=69)

# **Building of Multi-label Neural Network using Tensorflow**

In [106]:
tf.random.set_seed(69)
model = Sequential([
    tf.keras.Input(shape=(len(features),)),
    Dense(25, activation='relu', name='L1'),
    Dense(10, activation='relu', name='L2'),
    Dense(7, activation='sigmoid', name='L3')
], name='model'
)
model.summary()

Model: "model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ L1 (Dense)                      │ (None, 25)             │           525 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L2 (Dense)                      │ (None, 10)             │           260 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L3 (Dense)                      │ (None, 7)              │            77 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 862 (3.37 KB)

 Trainable params: 862 (3.37 KB)

 Non-trainable params: 0 (0.00 B)

In [107]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(
    X_train,y_train,
    epochs=40,
    validation_data=(X_test, y_test)
)

Epoch 1/40
481/481 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3671 - loss: 0.4569 - val_accuracy: 0.5008 - val_loss: 0.2685
Epoch 2/40
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5162 - loss: 0.2589 - val_accuracy: 0.5286 - val_loss: 0.2529
Epoch 3/40
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5340 - loss: 0.2474 - val_accuracy: 0.5315 - val_loss: 0.2478
Epoch 4/40
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5421 - loss: 0.2433 - val_accuracy: 0.5343 - val_loss: 0.2452
Epoch 5/40
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5462 - loss: 0.2410 - val_accuracy: 0.5382 - val_loss: 0.2433
Epoch 6/40
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5514 - loss: 0.2395 - val_accuracy: 0.5388 - val_loss: 0.2418
Epoch 7/40
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5537 - loss: 0.2384 - val_accuracy: 0.5419 - val_loss: 0.2407
Epoch 8/40
481/481 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5548 - loss: 0.2374 - val_accuracy: 0.

In [108]:
pred = pd.read_csv('/kaggle/input/playground-series-s4e3/test.csv')

feature_engineer(pred)
pred_scaled = ss.fit_transform(pred[features])
pred_scaled



array([[-0.27081003,  0.10284675,  1.03830143, ..., -0.42298531,
         0.82694441,  0.85872906],
       [-0.31873339, -0.35385755, -0.74400593, ..., -0.87379527,
         0.82694441,  0.85872906],
       [-0.33931858, -0.37146281, -0.90213962, ...,  0.02247824,
         0.82694441, -1.21042366],
       ...,
       [ 3.46059042,  3.17023692, -0.13332899, ...,  2.0074963 ,
         0.82694441, -1.21042366],
       [-0.32375762, -0.35037997, -0.59631249, ..., -0.42298531,
         0.82694441,  0.85872906],
       [ 0.80900954,  0.75679514,  0.00920998, ...,  1.7910733 ,
         0.82694441,  0.85872906]])

In [109]:
result = model.predict(pred_scaled)
submission = pd.DataFrame(result, columns = y_train.columns)
submission['id'] = pred.id


submission.to_csv('submission.csv', index=False)

submission.head()

401/401 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults,id
0,0.537360,0.002913,0.001200,0.000175,0.045406,0.085763,0.324428,19219
1,0.273212,0.013724,0.015707,0.000899,0.098851,0.141567,0.436656,19220
2,0.010941,0.086040,0.078291,0.000460,0.005517,0.356818,0.497201,19221
3,0.359510,0.002058,0.002716,0.000314,0.034762,0.215764,0.461360,19222
4,0.012333,0.001664,0.005166,0.000707,0.001785,0.508970,0.408603,19223
